In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from noisy_bn import NoisyBatchNormalization

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [4]:
train_images.shape

(60000, 28, 28)

In [31]:
# model 1
tf.keras.backend.clear_session()
input_node = keras.layers.Input((28,28))
expand = keras.layers.Lambda(lambda x: tf.expand_dims(tf.cast(x, tf.float32) / 255., axis=-1))(input_node)

conv1 = keras.layers.Conv2D(32, (3,3))(expand)
batch1 = keras.layers.BatchNormalization()(conv1)
pool1 = keras.layers.MaxPool2D((2,2))(batch1)

conv2 = keras.layers.Conv2D(32, (3,3))(pool1)
batch2 = keras.layers.BatchNormalization()(conv2)
pool2 = keras.layers.MaxPool2D((2,2))(batch2)

conv3 = keras.layers.Conv2D(32, (3,3))(pool2)
batch3 = keras.layers.BatchNormalization()(conv3)
pool3 = keras.layers.MaxPool2D((2,2))(batch3)

av1 = keras.layers.GlobalAveragePooling2D()(pool3)

h1 = keras.layers.Dense(512)(av1)
batch4 = keras.layers.BatchNormalization()(h1)
relu1 = keras.layers.Activation('relu')(batch4)

output_h = tf.keras.layers.Dense(10)(relu1)
output = tf.keras.layers.Activation('softmax')(output_h)



In [9]:
# model 2
tf.keras.backend.clear_session()
# input_node = keras.layers.Input((28,28))
# expand = keras.layers.Lambda(lambda x: tf.expand_dims(tf.cast(x, tf.float32) / 255., axis=-1))(input_node)

# conv1 = keras.layers.Conv2D(32, (3,3))(expand)
# batch1 = NoisyBatchNormalization(0)(conv1)
# pool1 = keras.layers.MaxPool2D((2,2))(batch1)

# conv2 = keras.layers.Conv2D(32, (3,3))(pool1)
# batch2 = NoisyBatchNormalization(0)(conv2)
# pool2 = keras.layers.MaxPool2D((2,2))(batch2)

# conv3 = keras.layers.Conv2D(32, (3,3))(pool2)
# batch3 = NoisyBatchNormalization(0)(conv3)
# pool3 = keras.layers.MaxPool2D((2,2))(batch3)

# av1 = keras.layers.GlobalAveragePooling2D()(pool3)

# h1 = keras.layers.Dense(512)(av1)
# batch4 = NoisyBatchNormalization(0)(h1)
# relu1 = keras.layers.Activation('relu')(batch4)

# output_h = tf.keras.layers.Dense(10, act)(relu1)
# output = tf.keras.layers.Activation('softmax')(output_h)


class MyModel(keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        
        self.pre_pro = keras.layers.Lambda(lambda x: tf.expand_dims(tf.cast(x, tf.float32) / 255., axis=-1))
        
        self.conv1 = keras.layers.Conv2D(32, (3,3))
        self.conv2 = keras.layers.Conv2D(32, (3,3))
        self.conv3 = keras.layers.Conv2D(32, (3,3))
        
        self.batch1 = NoisyBatchNormalization(0)
        self.batch2 = NoisyBatchNormalization(0)
        self.batch3 = NoisyBatchNormalization(0)
        self.batch4 = NoisyBatchNormalization(0)
        
        self.pool = keras.layers.MaxPool2D((2,2))
        
        self.globalav = keras.layers.GlobalAveragePooling2D()
        
        self.dense1 = keras.layers.Dense(512)
        self.dense2 = keras.layers.Dense(10, activation='softmax')
        self.relu1 = keras.layers.Activation('relu')
        
    def call(self, inputs):
        
        pro_inp = self.pre_pro(inputs)
        
        print(pro_inp.shape)
        
        conv1 = self.conv1(pro_inp)
        batch1 = self.batch1(conv1)
        pool1 = self.pool(batch1)
        
        conv2 = self.conv1(pool1)
        batch2 = self.batch1(conv2)
        pool2 = self.pool(batch2)

        
        conv3 = self.conv1(pool2)
        batch3 = self.batch1(conv3)
        pool3 = self.pool(batch3)
        
        av = self.globalav(pool3)
        
        dense1 = self.dense1(av)
        batch4 = self.batch4(dense1)
        relu1 = self.relu1(batch4)
        
        pred = self.dense2(relu1)
        
        return pred
        


In [10]:
model = MyModel()

In [11]:
model(train_images[:3])

(3, 28, 28, 1)
[0, 1, 2, 3] 3


UnimplementedError: Generic conv implementation does not support grouped convolutions for now. [Op:Conv2D]

In [35]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [36]:
from tensorflow.keras.utils import to_categorical
model.fit(train_images, to_categorical(train_labels), batch_size=32, epochs=5,validation_data=(test_images, to_categorical(test_labels)), verbose=1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 30s 503us/sample - loss: 0.5211 - accuracy: 0.8121 - val_loss: 0.4071 - val_accuracy: 0.8541
Epoch 2/5
60000/60000 [==============================] - 30s 498us/sample - loss: 0.3902 - accuracy: 0.8580 - val_loss: 0.4070 - val_accuracy: 0.8509
Epoch 3/5
60000/60000 [==============================] - 30s 497us/sample - loss: 0.3522 - accuracy: 0.8726 - val_loss: 0.3816 - val_accuracy: 0.8634
Epoch 4/5
60000/60000 [==============================] - 30s 498us/sample - loss: 0.3252 - accuracy: 0.8813 - val_loss: 0.4067 - val_accuracy: 0.8575
Epoch 5/5
60000/60000 [==============================] - 30s 496us/sample - loss: 0.3026 - accuracy: 0.8894 - val_loss: 0.3436 - val_accuracy: 0.8753


In [ ]:
tf.keras.layers.BatchNormalization()

In [16]:
class TestLayer(tf.keras.layers.Layer):
    def __init__(self, units, input_dim):
        super(TestLayer, self).__init__()
        self.w = self.add_weight(shape=(input_dim, units),
                             initializer=tf.zeros_initializer,
                             trainable=True)
        
        
    def call(self, inputs, training=None):
        self.w = self.w + 1
        return inputs

In [17]:
test_layer = TestLayer(32, 32)

In [18]:
input_const = tf.constant([1,2,3])
output_const = test_layer(input_const)

In [19]:
test_layer.w

<tf.Tensor: id=26, shape=(32, 32), dtype=float32, numpy=
array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)>